# Scraper para o site da Hemeroteca Digital de Lisboa

Esse notebook busca coletar os dados dos periódicos digitalizados e disponíveis no site da Hemeroteca Digital de Lisboa. Os dados serão organizados em um *dataframe* e exportados para um arquivo csv.

Os dados coletados são:

- **Título**: Título do periódico
- **Autoria**: Autoria do periódico
- **Período**: Período disponível do periódico
- **Link**: Link para o periódico

In [ ]:
# importar bibliotecas
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [ ]:
# definir lista de letras que foram o índice
letras = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J','L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U','V', 'X', 'Z']

# definir url base
url_base = 'https://hemerotecadigital.cm-lisboa.pt/Indice/Indice'

# final da url
url_final = '.htm'

In [ ]:
# definir lista de urls completa
urls = [url_base + letra + url_final for letra in letras]
print(urls)

In [ ]:
# função para criar objeto soup
def criar_soup(url):
    html = requests.get(url)
    soup = bs(html.content, 'html.parser')
    return soup

In [ ]:
# find all a href that ends with .htm and start with ../Periodicos or ../OBRAS
def encontrar_links(soup):
    links = soup.find_all('a', href=True)
    links = [link['href'] for link in links if link['href'].endswith('.htm') and (link['href'].startswith('../Periodicos') or link['href'].startswith('../OBRAS'))]
    # replace ../ with https://hemerotecadigital.cm-lisboa.pt/
    links = [link.replace('../', 'https://hemerotecadigital.cm-lisboa.pt/') for link in links]
    return links

In [ ]:
# loop com as urls
links_geral = []
for url in urls:
    soup = criar_soup(url)
    links = encontrar_links(soup)
    links_geral.extend(links)
print(len(links_geral))

In [ ]:
# loop nos links
for link in links_geral:
    soup = criar_soup(link)
    print(soup.title.text)